In [ ]:
# Extract JSON from Camden Trees API and store it in a table in UCL database
!pip install pandas
!pip install sodapy

import pandas as pd
from sodapy import Socrata
import requests
import pymysql
import os
import json

#!!!!this is my registered account with Camden COuncil, if prompted a lot, they will lock it !!!
client = Socrata("opendata.camden.gov.uk","5YbHlvnsIgblrrY3kSuIU9WZ0", username = "ucfnmoa@ucl.ac.uk",password = "Archsara13" )

# increased the limit to cover all the records, returned as JSON from API / converted to Python list of dictionaries by sodapy.
r = client.get("csqp-kdss", limit=24000)

def validate_string(val):
    if val != None:
        if type(val) is int:
            #for x in val:
            #   print(x)
            return str(val).encode('utf-8')
        else:
            return val
        
# connect with UCL database, enter your user, password and database from week 1       
mydb = pymysql.connect(host='dev.spatialdatacapture.org',
user='',
passwd='',
db='')

cursor = mydb.cursor()

for i,item in enumerate(r):
    latitude=validate_string(item.get("latitude", None))
    longitude = validate_string(item.get("longitude", None))
    cursor.execute("CREATE TABLE trees_in_Camden (lat DOUBLE , lon DOUBLE)")
    cursor.execute("INSERT INTO trees_in_Camden (lat,lon) VALUES (%s,%s)", (latitude, longitude))
    
#close the connection to the database.
mydb.commit()
cursor.close()
print("Done")

# it took about 7 min
#check in mySQL your database, you can find the trees in camden table with all the records!